In [1]:
using LinearAlgebra, Test

In [ ]:
A = [1 2 3; 
     4 5 6;
     7 8 9; 
     10 11 12]
rank(A)

In [ ]:
function householderreflection(x)
    T = float(eltype(x))
    y = Vector{T}(x)
    y[1] += norm(x)*sign(x[1])
    w = y/norm(y)
    I - 2w*w' # I represents identity matrix on any dimension
end

In [ ]:
A = [1 2 3; 
     4 5 6;
     7 8 9; 
     10 11 13]

m,n = size(A)

𝐚₁ = A[:,1]
Q₁ = householderreflection(𝐚₁)
Q₁*𝐚₁ # first column made "sparse"

In [ ]:
Q₁A = Q₁*A # first column made "sparse"
α₁, 𝐰₁ = Q₁A[1,1],Q₁A[1,:]

R₁ = Q₁ * R₁
A ≈ Q₁ * R₁ # partial-QR

In [ ]:
A₂ = Q₁A[2:end,2:end]
𝐚₂ = A₂[:,1]
α₂, 𝐰₂ = R₂[1,1],R₂[1,:]
Q₂ = householderreflection(𝐚₂)

Q₂ * A₂

In [ ]:
Q̃₂ = Matrix(1.0I, m, m)
Q̃₂[2:end, 2:end] = Q₂

R₂ = Q̃₂*Q₁*A # first two columns made sparse

A ≈  Q₁*Q̃₂ * R₂ # partial-QR

In [ ]:
A₃ = (Q₂ * A₂)[2:end,2:end]
𝐚₃ = A₃[:,1]

Q₃ = householderreflection(𝐚₃)
Q̃₃ = Matrix(1.0I, m, m)
Q̃₃[3:end, 3:end] = Q₃

R = Q̃₃ * Q̃₂*Q₁*A # first two columns made sparse

Q = Q₁*Q̃₂*Q̃₃

@test A ≈ Q*R
@test Q'Q ≈ I

In [ ]:
Q₁ * Q̃₂ # Q₂ acts on just columns 2:n
Q = copy(Q₁)
Q[:,2:end] = Q[:,2:end] * Q₂
@test Q == Q₁ * Q̃₂

Q₁ * Q̃₂ *Q̃₃ # Q₃ acts on just columns 3:n
Q[:,3:end] = Q[:,3:end] * Q₃
@test Q ≈ Q₁ * Q̃₂ *Q̃₃

@test A ≈ Q*R

In [ ]:
# algorithm
function householderqr(A)

    m,n = size(A)

    T = float(eltype(A))
    R = zeros(T, m, n)
    Q = Matrix{T}(I, m, m)

    Aⱼ = A


    for j = 1:n
        𝐚ⱼ = Aⱼ[:,1]
        Qⱼ = householderreflection(𝐚ⱼ)
        QⱼAⱼ = Qⱼ*Aⱼ # WARNING!!! O(n*m^2) operations! should be O(n*m)
        α,𝐰 = QⱼAⱼ[1,1],QⱼAⱼ[1,2:end]
        Aⱼ₊₁ = QⱼAⱼ[2:end,2:end]

        # populate R
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        # update Q
        Q[:,j:end] = Q[:,j:end] * Qⱼ

        Aⱼ = Aⱼ₊₁
    end

    Q,R
    
end